In [12]:
import redis
import numpy as np
import pandas as pd

In [13]:
hostname = 'redis-16143.c12.us-east-1-4.ec2.cloud.redislabs.com'
port = '16143'
password = 'IuUBnaQGK79atnDHJpOK9qCgcgO0Xq7V'

In [14]:
r = redis.StrictRedis(host=hostname,
               port= port, password=password)

In [15]:
r.ping()

True

In [16]:
# load the data numpy.zip

In [17]:
data_restored = np.load('face_dataframe_compressed.npz', allow_pickle=True)
x_values = data_restored['arr_0']
columns = data_restored['arr_1']

df = pd.DataFrame(x_values, columns=columns)
df

,Name,Role,Facial Features,cosine
0,Angelina Jolie,student,"[0.43239492, 1.0748208, -0.12874822, -1.833274...",-0.055578
1,Barack_Obama,teacher,"[0.94851226, -0.5582868, 0.09495868, -1.279620...",-0.109103
2,Chris_Evans,student,"[0.7200426, -1.4685223, -0.7779212, -1.460148,...",0.0282
3,Morgan_Freeman,teacher,"[0.48528442, -0.619405, -0.4070305, 1.0686811,...",0.07118
4,Scarlett Johansson,student,"[0.19131908, -0.48838148, -1.690325, 0.4423288...",0.013093


In [18]:
# genereate unique key name + role for storing data in redis
df['name_role'] = df['Name']+'#'+df['Role']

In [19]:
df

,Name,Role,Facial Features,cosine,name_role
0,Angelina Jolie,student,"[0.43239492, 1.0748208, -0.12874822, -1.833274...",-0.055578,Angelina Jolie#student
1,Barack_Obama,teacher,"[0.94851226, -0.5582868, 0.09495868, -1.279620...",-0.109103,Barack_Obama#teacher
2,Chris_Evans,student,"[0.7200426, -1.4685223, -0.7779212, -1.460148,...",0.0282,Chris_Evans#student
3,Morgan_Freeman,teacher,"[0.48528442, -0.619405, -0.4070305, 1.0686811,...",0.07118,Morgan_Freeman#teacher
4,Scarlett Johansson,student,"[0.19131908, -0.48838148, -1.690325, 0.4423288...",0.013093,Scarlett Johansson#student


In [20]:
# convert records to dictionary to store in redis
records = df[['name_role','Facial Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
Using Redis Hashes
keyname = 'school:register'

In [21]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial Features']
    
    # convert numpy array to bytes
    vector_bytes = vector.tobytes()
    
    # save data to redis cloud
    r.hset(name='school:register',key=name_role,value=vector_bytes)

In [22]:
vector.dtype

dtype('float32')

## Retrive Records from Redis

In [23]:
decoded_dictionary = r.hgetall(name='school:register')
decoded_series = pd.Series(decoded_dictionary)
decoded_series

b'Morgan_Freeman#teacher'        b'3w\xf8>S\x91\x1e\xbfMf\xd0\xbe\x8b\xca\x88?\...
b'Angelina Jolie#student'        b'\xdeb\xdd>\xba\x93\x89?\x93\xd6\x03\xbe\xba\...
b'Barack_Obama#teacher'          b'\xb3\xd1r?\xe2\xeb\x0e\xbf\xb2y\xc2=\x9a\xca...
b'Chris_Evans#student'           b'\xb6T8?\x8a\xf8\xbb\xbf\xd8%G\xbf!\xe6\xba\x...
b'Scarlett Johansson#student'    b'&\xe9C>#\r\xfa\xbe\x92\\\xd8\xbf\xedx\xe2>\x...
dtype: object

In [24]:
# Convert each element of retrive_series from bytes to 32-bit float NumPy arrays
decoded_series = decoded_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

# Retrieve the index of retrive_series
index = decoded_series.index

# Print the retrieved index
print(index)

# Convert the index elements from bytes to strings
index = list(map(lambda x: x.decode(), index))



Index([    b'Morgan_Freeman#teacher',     b'Angelina Jolie#student',
             b'Barack_Obama#teacher',        b'Chris_Evans#student',
       b'Scarlett Johansson#student'],
      dtype='object')


In [25]:
# Print the modified index after decoding
print(index)

['Morgan_Freeman#teacher', 'Angelina Jolie#student', 'Barack_Obama#teacher', 'Chris_Evans#student', 'Scarlett Johansson#student']


In [26]:
decoded_series.index = index

In [27]:
decoded_df = retrive_series.to_frame().reset_index()
decoded_df.columns = ['name_role', 'facial_features']
decoded_df

NameError: name 'retrive_series' is not defined